In [2]:
import numpy as np
import gym
from gym import spaces, logger
from gym.utils import seeding
import copy
import matplotlib.pyplot as plt
from collections import deque
from pyomo.environ import *
from pyomo.opt import SolverFactory
import or_gym
%matplotlib inlinelearn

In [3]:
env = gym.make('Knapsack-v0')

In [80]:
class KnapsackEnv(gym.Env):
    
    def __init__(self):
        self.item_weights = [1, 2, 3, 6, 10, 18]
        self.item_values = [0, 1, 3, 14, 20, 100]
        self.item_numbers = np.arange(len(self.item_weights))
        self.N = len(self.item_weights)
        self.max_weight = 15
        self.current_weight = 0
        
        self.action_space = spaces.Discrete(self.N)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(self.N),
            spaces.Discrete(self.N),
            spaces.Discrete(1)))
        
        self.seed()
        self.reset()
        
    def step(self, item):
        # Check that item will fit
        if self.item_weights[item] + self.current_weight <= self.max_weight:
            self.current_weight += self.item_weights[item]
            reward = self.item_values[item]
            if self.current_weight == self.max_weight:
                done = True
            else:
                done = False
        else:
            # End trial if over weight
            reward = 0
            done = True
            
        self._update_state()
        return self.state, reward, done, {}
    
    def _get_obs(self):
        return self.state
    
    def _update_state(self):
        self.state = (self.item_weights,
                      self.item_values, 
                      self.current_weight)
    
    def reset(self):
        self.current_weight = 0
        self._update_state()
        return self.state
    
    def sample_action(self):
        return np.random.choice(self.item_numbers)

In [81]:
env = KnapsackEnv()
env.step(2)

(([1, 2, 3, 6, 10, 18], [0, 1, 3, 14, 20, 100], 3), 3, False, {})

# Solve KP with DP

We can test this model by solving it with dynamic programming approaches.

Thikning about this carefully, we see that, for the simple dynamic programming approach to the unbounded problem, there are only as many states as the carrying capacity divided by the smallest weight. In the simple case above, we have a max capacity of 15, and the smallest item we have has a weight of 1. This gives us 15 states. This becomes much more complicated when we have a bounded problem such that the state is defined by the weight and the remaining number of items to pack, so we'll leave that for later.

## Policy Iteration

Value iteration was easy to solve for and determine the values. So let's move on to policy iteration to get a better understanding of how this works.

We start with a random policy where all actions at each state are equally probable. We then alternate between evaluating the policy and improving the policy.

In [4]:
def policy_evaluation(env, policy, gamma):
    # Value iteration
    v = np.zeros(env.max_weight + 1)

    # Terminal state values are 0
    delta = 1e-5
    delta_t = 1
    k = 0
    while delta_t > delta:
        v_new = np.zeros(v.shape)
        for i in range(v_new.shape[0]):
            # Check for terminal state
            if i == env.max_weight:
                continue
            else:
                for action in env.item_numbers:
                    if i + env.item_weights[action] <= env.max_weight:
                        s_1 = i + env.item_weights[action]
                        r = env.item_values[action]
                    else:
                        s_1 = env.N
                        r = 0
                    prob = policy[i, action]
                    v_new[i] += prob * (r + gamma * v[s_1])

        k += 1
        delta_t = np.sum(np.abs(v - v_new))
        v = v_new.copy()
    return v

def policy_improvement(env, policy, values, gamma):
    new_policy = np.zeros(policy.shape)
    
    for i in range(new_policy.shape[0]):
        for j in range(new_policy.shape[1]):
            # Get the action values
            action_vals = []
            for action in env.item_numbers:
                # Update state and take step
                env.current_weight = i
                env._get_obs()
                s_1, r, done, _ = env.step(action)
                val = r + gamma * (values[i])
                action_vals.append([val, action])
                
            # Select the best action
            action_vals = np.array(action_vals)
            best_actions = np.argwhere(
                action_vals[:,0]==np.max(action_vals[:, 0])).flatten()
            # Randomize in cases where multiple maximums exist
            if len(best_actions) > 1:
                best_actions = np.random.choice(best_actions)
            new_policy[i, best_actions] = 1

    num_policy_changes = np.sum(policy != new_policy).astype(int)
    return new_policy, num_policy_changes

In [5]:
def play_policy(env, policy):
    state = env.reset()[-1]
    done = False
    rewards = []
    actions = []
    while done == False:
        action = np.argmax(policy[state])
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        actions.append(action)
        state = next_state[-1]
    return np.array(rewards), np.array(actions)

In [6]:
# Initialize Policy
policy = np.ones((env.max_weight + 1, env.N)) * 1 / env.N
env.reset()
gamma = 1
k = 0
reward_deque = deque(maxlen=3)
action_deque = deque(maxlen=3)
while True:
    values = policy_evaluation(env, policy, gamma)
    policy, num_changes = policy_improvement(env, policy, values, gamma)
    k += 1
    if num_changes == 0:
        break
    # After so long, it looks like these policies are only updating
    # at the very end where it doesn't matter. We'll play the env
    # with the policies to avoid an infinite loop. If 3 in a row are
    # identical, then we'll call it quits
    rewards, actions = play_policy(env, policy)
    reward_deque.append(rewards)
    action_deque.append(actions)
    if len(reward_deque) == 3:
        converge_r = all([np.allclose(reward_deque[i-1], reward_deque[i])
            for i, j in enumerate(reward_deque)])
        converge_a = all([np.allclose(action_deque[i-1], action_deque[i])
            for i, j in enumerate(action_deque)])
        if converge_r and converge_a:
            print("Converged after {} iterations".format(k))
            break

Converged after 3 iterations


In [7]:
action_deque[0]

array([4, 2, 1], dtype=int64)

## MILP of Model

$$\textrm{max} \; z = \sum_i v_i x_i$$

$$\textrm{s.t.} \; \sum_i w_i x_i \leq W$$

$$x_i \in [0, 1]$$
$$v_i, w_i \in \rm I\!R^+$$

In [8]:
# Initialize model
m = ConcreteModel()

# Sets, parameters, and variables
m.W = env.max_weight
m.i = Set(initialize=env.item_numbers)
m.w = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_weights)})
m.v = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_values)})
m.x = Var(m.i, within=Binary)

In [9]:
@m.Constraint(m.i)
def weight_constraint(m, i):
    return sum(m.w[i] * m.x[i] for i in m.i) - m.W <= 0

m.obj = Objective(expr=(
    sum([m.v[i] * m.x[i] for i in m.i])),
    sense=maximize)

In [10]:
solver = SolverFactory('glpk')
results = solver.solve(m, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\U755275\AppData\Local\Temp\tmpfe5eyz0f.glpk.raw --wglp C:\Users\U755275\AppData\Local\Temp\tmpxtl8b7lk.glpk.glp
 --cpxlp C:\Users\U755275\AppData\Local\Temp\tmpn9s7hjk2.pyomo.lp
Reading problem data from 'C:\Users\U755275\AppData\Local\Temp\tmpn9s7hjk2.pyomo.lp'...
C:\Users\U755275\AppData\Local\Temp\tmpn9s7hjk2.pyomo.lp:78: warning: lower bound of variable 'x1' redefined
C:\Users\U755275\AppData\Local\Temp\tmpn9s7hjk2.pyomo.lp:78: warning: upper bound of variable 'x1' redefined
7 rows, 7 columns, 37 non-zeros
6 integer variables, all of which are binary
84 lines were read
Writing problem data to 'C:\Users\U755275\AppData\Local\Temp\tmpxtl8b7lk.glpk.glp'...
67 lines were written
GLPK Integer Optimizer, v4.65
7 rows, 7 columns, 37 non-zeros
6 integer variables, all of which are binary
Preprocessing...
6 constraint coefficient(s) were reduced
6 rows, 5 columns, 30 non-zeros
5 integer variables

In [11]:
m.obj.expr()

24.0

In [12]:
[i for i in m.i if m.x[i]==1]

[1, 2, 4]